<a href="https://colab.research.google.com/github/yulinlina/MedMnist/blob/RetinaMNIST/ConvNeXt_for_MedMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use this notebook to finetune a ConvNeXt-tiny model on CIFAR 10 dataset. The [official ConvNeXt repository](https://github.com/facebookresearch/ConvNeXt) is instrumented with [Weights and Biases](https://wandb.ai/site). You can now easily log your train/test metrics and version control your model checkpoints to Weigths and Biases

# ⚽️ Installation and Setup

The following installation instruction is based on [INSTALL.md](https://github.com/facebookresearch/ConvNeXt/blob/main/INSTALL.md) provided by the official ConvNeXt repository. 

In [1]:
#运行前先在"代码执行程序"中选择"更改运行时类型"为GPU
!pip install -qq torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -qq wandb timm==0.4.12 six tensorboardX

ERROR: Could not find a version that satisfies the requirement torch==1.8.0+cu111 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2)
ERROR: No matching distribution found for torch==1.8.0+cu111
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.0 MB/s eta 0:00

Download the official ConvNeXt respository. 

In [2]:
!git clone https://github.com/facebookresearch/ConvNeXt

Cloning into 'ConvNeXt'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 252 (delta 129), reused 193 (delta 111), pack-reused 3
Receiving objects: 100% (252/252), 69.37 KiB | 2.89 MiB/s, done.
Resolving deltas: 100% (129/129), done.


# 🏀 Download the Dataset

We will be finetuning on CIFAR-10 dataset. To use any custom dataset (CIFAR-10 here) the format of the dataset should be as shown below:

```
/path/to/dataset/
  train/
    class1/
      img1.jpeg
    class2/
      img2.jpeg
  val/
    class1/
      img3.jpeg
    class2/
      img4.jpeg
```



In [3]:
!pip install --upgrade git+https://github.com/MedMNIST/MedMNIST.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MedMNIST/MedMNIST.git to /tmp/pip-req-build-r_2s8xbx
  Running command git clone --filter=blob:none --quiet https://github.com/MedMNIST/MedMNIST.git /tmp/pip-req-build-r_2s8xbx
  Resolved https://github.com/MedMNIST/MedMNIST.git to commit 18a7564bc1fc3c68adbfeac7590d3949fe91467b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for medmnist: filename=medmnist-2.2.2-py3-none-any.whl size=21964 sha256=e257697b816d2a4fca67099001010a97395d5d1701209091272e2435f7e84168
  Stored in directory: /tmp/pip-ephem-wheel-cache-t5xc8hgw/wheels/71/ef/8a/0e687fea04ab72752eb9d88f3e89bf5efdc98dd494e55d0b01
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=b05b401f112a4baf5b7fee0ad9a39f26850285360ffc0c9e445086bc1d1b036f

In [4]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator
data_flag = 'retinamnist'
# data_flag = 'breastmnist'
download = True
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])
# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

100%|██████████| 3291041/3291041 [00:06<00:00, 530504.18it/s]

Using downloaded and verified file: /root/.medmnist/retinamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz


In [5]:
!python -m medmnist save --flag=retinamnist --folder=MedMNIST/ --postfix=jpeg

Saving retinamnist train...
100% 1080/1080 [00:00<00:00, 3904.58it/s]
Saving retinamnist val...
100% 120/120 [00:00<00:00, 4100.57it/s]
Saving retinamnist test...
100% 400/400 [00:00<00:00, 3909.28it/s]


In [6]:
#以下是数据集格式整理脚本，目的是把数据集变成类似CIFAR-10的格式，忽略mv: cannot stat 'val802_6.jpeg': No such file or directory等等输出即可
%cd /content/MedMNIST/retinamnist/
%mkdir train val test
%cd /content/MedMNIST/retinamnist/train/
%mkdir class1 class2 class3 class4 class5
%cd /content/MedMNIST/retinamnist/test/
%mkdir class1 class2 class3 class4 class5
%cd /content/MedMNIST/retinamnist/val/
%mkdir class1 class2 class3 class4 class5
%cd /content/MedMNIST/retinamnist/
%mv train{0..1079}_0.jpeg /content/MedMNIST/retinamnist/train/class1/
%mv train{0..1079}_1.jpeg /content/MedMNIST/retinamnist/train/class2/
%mv train{0..1079}_2.jpeg /content/MedMNIST/retinamnist/train/class3/
%mv train{0..1079}_3.jpeg /content/MedMNIST/retinamnist/train/class4/
%mv train{0..1079}_4.jpeg /content/MedMNIST/retinamnist/train/class5/
%cd /content/MedMNIST/retinamnist/
%mv test{0..399}_0.jpeg /content/MedMNIST/retinamnist/test/class1/
%mv test{0..399}_1.jpeg /content/MedMNIST/retinamnist/test/class2/
%mv test{0..399}_2.jpeg /content/MedMNIST/retinamnist/test/class3/
%mv test{0..399}_3.jpeg /content/MedMNIST/retinamnist/test/class4/
%mv test{0..399}_4.jpeg /content/MedMNIST/retinamnist/test/class5/
%cd /content/MedMNIST/retinamnist/
%mv val{0..119}_0.jpeg /content/MedMNIST/retinamnist/val/class1/
%mv val{0..119}_1.jpeg /content/MedMNIST/retinamnist/val/class2/
%mv val{0..119}_2.jpeg /content/MedMNIST/retinamnist/val/class3/
%mv val{0..119}_3.jpeg /content/MedMNIST/retinamnist/val/class4/
%mv val{0..119}_4.jpeg /content/MedMNIST/retinamnist/val/class5/

流式输出内容被截断，只能显示最后 5000 行内容。
mv: cannot stat 'train915_1.jpeg': No such file or directory
mv: cannot stat 'train916_1.jpeg': No such file or directory
mv: cannot stat 'train917_1.jpeg': No such file or directory
mv: cannot stat 'train918_1.jpeg': No such file or directory
mv: cannot stat 'train919_1.jpeg': No such file or directory
mv: cannot stat 'train920_1.jpeg': No such file or directory
mv: cannot stat 'train921_1.jpeg': No such file or directory
mv: cannot stat 'train922_1.jpeg': No such file or directory
mv: cannot stat 'train923_1.jpeg': No such file or directory
mv: cannot stat 'train924_1.jpeg': No such file or directory
mv: cannot stat 'train925_1.jpeg': No such file or directory
mv: cannot stat 'train926_1.jpeg': No such file or directory
mv: cannot stat 'train927_1.jpeg': No such file or directory
mv: cannot stat 'train928_1.jpeg': No such file or directory
mv: cannot stat 'train929_1.jpeg': No such file or directory
mv: cannot stat 'train930_1.jpeg': No such file or directo

# 🏈 Download Pretrained Weights

We will be finetuning the ConvNeXt Tiny model pretrained on ImageNet 1K dataset.

In [7]:
#先检查一下/content/MedMNIST/retinamnist文件夹是否把所有图片归类到train val test
%cd /content/ConvNeXt/
#下面是下载预训练模型，需要用到在Imagenet-1k上预训练模型，否则效果不好(可以去掉试试)
!wget https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth

/content/ConvNeXt
--2023-05-23 07:19:27--  https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 99.84.160.108, 99.84.160.61, 99.84.160.80, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|99.84.160.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 265112135 (253M) [binary/octet-stream]
Saving to: ‘convnext_small_22k_224.pth’

convnext_small_22k_ 100%[===================>] 252.83M  32.1MB/s    in 8.2s    

2023-05-23 07:19:35 (30.7 MB/s) - ‘convnext_small_22k_224.pth’ saved [265112135/265112135]



# 🎾 Train with Weights and Biases

If you want to log the train and evaluation metrics using Weights and Biases pass `--enable_wandb true`. 

You can also save the finetuned checkpoints as version controlled W&B [Artifacts](https://docs.wandb.ai/guides/artifacts) if you pass `--wandb_ckpt true`.



In [8]:
!python main.py --epochs 30 \
                --model convnext_small \
                --data_set image_folder \
                --data_path /content/MedMNIST/retinamnist/train \
                --eval_data_path /content/MedMNIST/retinamnist/test \
                --nb_classes 5 \
                --num_workers 8 \
                --warmup_epochs 0 \
                --save_ckpt true \
                --output_dir model_ckpt \
                --cutmix 0 \
                --mixup 0 --lr 4e-4 \
                --enable_wandb true --wandb_ckpt true \
                --finetune convnext_small_22k_224.pth 

Not using distributed mode
Namespace(batch_size=64, epochs=30, update_freq=1, model='convnext_small', drop_path=0, input_size=224, layer_scale_init_value=1e-06, model_ema=False, model_ema_decay=0.9999, model_ema_force_cpu=False, model_ema_eval=False, opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=None, momentum=0.9, weight_decay=0.05, weight_decay_end=None, lr=0.0004, layer_decay=1.0, min_lr=1e-06, warmup_epochs=0, warmup_steps=-1, color_jitter=0.4, aa='rand-m9-mstd0.5-inc1', smoothing=0.1, train_interpolation='bicubic', crop_pct=None, reprob=0.25, remode='pixel', recount=1, resplit=False, mixup=0.0, cutmix=0.0, cutmix_minmax=None, mixup_prob=1.0, mixup_switch_prob=0.5, mixup_mode='batch', finetune='convnext_small_22k_224.pth', head_init_scale=1.0, model_key='model|module', model_prefix='', data_path='/content/MedMNIST/retinamnist/train', eval_data_path='/content/MedMNIST/retinamnist/test', nb_classes=5, imagenet_default_mean_and_std=True, data_set='image_folder', output_dir='mo

In [ ]:
#装载Google dirve
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#保存
%cd /content
%cp -r MedMNIST drive/MyDrive/MedMNIST
%cp -r ConVNeXt drive/MyDrive/ConVNeXt

/content


# 🏐 Conclusion

* **The above setting gives a top-1 accuracy of ~95%.**
* The ConvNeXt repository comes with modern training regimes and is easy to finetune on any dataset. 
* The finetune model achieves competitive results. 

* By passing two arguments you get the following:

  * Repository of all your experiments (train and test metrics) as a [W&B Project](https://docs.wandb.ai/ref/app/pages/project-page). You can easily compare experiments to find the best performing model.
  * Hyperparameters (Configs) used to train individual models. 
  * System (CPU/GPU/Disk) metrics.
  * Model checkpoints saved as W&B Artifacts. They are versioned and easy to share. 

  Check out the associated [W&B run page](https://wandb.ai/ayut/convnext/runs/16vi9e31). $→$